For one language English text and Shlokas 

In [2]:
import numpy as np
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string
import pickle


In [3]:
paras_e = open('Geeta_English.txt', encoding="UTF8").read().split("\n")


#preprocessing

In [4]:
def preprocess_text(text):
   
    tokens = word_tokenize(text.lower())

    # Remove punctuations
    tokens = [token for token in tokens if token not in string.punctuation]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Stem the words
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    return tokens

Creating Posting list

In [5]:
pos_list = {}
for idx, para in enumerate(paras_e):
    words = preprocess_text(para)
    
    for word in words:
        if word in pos_list.keys():
            if idx in pos_list[word].keys():
                pos_list[word][idx] += 1
            else:
                pos_list[word][idx] = 1
        else:
            temp = {idx : 1}
            pos_list[word] = temp.copy()

In [6]:
file_path ='Geeta_English.txt'



In [7]:
with open('posting_list.pickle', 'wb') as file:
    pickle.dump(pos_list, file)

Implementing BM25

In [8]:
def BM25(query, w_coll, l = 5, b = 0.75, k = 2):
    q_tokens = preprocess_text(query)
    lengths = {}
    N = len(paras_e)
    avg_len = 0
    for idx, para in enumerate(paras_e):
        lengths[idx] = len(para)             #cal no of words of each file
        avg_len += lengths[idx]
    avg_len /= N
    #Calculate idf of each token
    idf = {}
    for word in np.unique(q_tokens):
        if word in w_coll.keys():
            df = len(w_coll[word].keys())
        else:
            df = 0
        idf[word] = np.log((N - df + 0.5) / (df + 0.5))
    score = {}
    for idx, para in enumerate(paras_e):
        s = 0
        for word in np.unique(q_tokens):
            tf = 0
            if word in w_coll.keys() and idx in w_coll[word].keys():
                tf = w_coll[word][idx]
            s += idf[word] * (tf * (k + 1)) / (k*(1 - b + b*lengths[idx]/avg_len) + tf)
        score[idx] = s
    return sorted(score, key = score.get, reverse=True)[:l]

Checking search function for BM25 model Prototype

In [9]:
slokas = open("Shlokas.txt",encoding="UTF-8").read().split("\n")

In [10]:
def search(query, w_col, lang, l = 5):
    
    tokens = preprocess_text(query)
    
            
    query = " ".join(tokens)
#     print(query)
    indxs = BM25(query, w_col, l)
    print(indxs)
    if lang == 'e':
        return [f'{slokas[idx]}{paras_e[idx]}' for idx in indxs], indxs
   

Evaluation 

In [11]:
query ="That gives itself to follow shows of sense"
search(query,pos_list,"e")

[30, 0, 1, 2, 3]


(['तस्य सञ्जनयन्हर्षं कुरुवृद्धः पितामहः ।Our battle shows where Bhishma holds command,',
  'धृतराष्ट्र उवाच । I',
  'धर्मक्षेत्रे कुरुक्षेत्रे समवेता युयुत्सवः ।Dhritirashtra:',
  'मामकाः पाण्डवाश्चैव किमकुर्वत सञ्जय ॥ Ranged thus for battle on the sacred plain--',
  'सञ्जय उवाच ।On Kurukshetra--say, Sanjaya! say'],
 [30, 0, 1, 2, 3])